# Module 3 Homework

In [1]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.graph_objects as go
import plotly.express as px

import time
from datetime import date
from pathlib import Path

# for graphs
import matplotlib.pyplot as plt

In [2]:
# dataset file
dir = "../03-modeling/content/"
file = "stocks_df_combined_2024_05_07.parquet.brotli"
dataset = Path(dir, file)

if not dataset.exists():
    # full dataset for 33 stocks
    !gdown https://drive.google.com/file/d/1kNWWPi49td0EZhmi6LzNCa2ssC5IUxHP/view?usp=sharing --fuzzy -O ../03-modeling/content/

df_full = pd.read_parquet(dataset, )

In [3]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 221142 entries, 0 to 5426
Columns: 202 entries, Open to growth_btc_usd_365d
dtypes: datetime64[ns](3), float64(128), int32(64), int64(5), object(2)
memory usage: 288.5+ MB


In [4]:
df_full.keys()

Index(['Open', 'High', 'Low', 'Close', 'Adj Close_x', 'Volume', 'Ticker',
       'Year', 'Month', 'Weekday',
       ...
       'growth_brent_oil_7d', 'growth_brent_oil_30d', 'growth_brent_oil_90d',
       'growth_brent_oil_365d', 'growth_btc_usd_1d', 'growth_btc_usd_3d',
       'growth_btc_usd_7d', 'growth_btc_usd_30d', 'growth_btc_usd_90d',
       'growth_btc_usd_365d'],
      dtype='object', length=202)

In [5]:
# features
GROWTH = [g for g in df_full.keys() if (g.find('growth_')==0)&(g.find('future')<0)]
OHLCV = ['Open','High','Low','Close','Adj Close_x','Volume']
CATEGORICAL = ['Month', 'Weekday', 'Ticker', 'ticker_type']
TO_PREDICT = [g for g in df_full.keys() if (g.find('future')>=0)]
TO_DROP = ['Year','Date','index_x', 'index_y', 'index', 'Quarter','Adj Close_y'] + CATEGORICAL + OHLCV

# manually defined features
CUSTOM_NUMERICAL = ['SMA10', 'SMA20', 'growing_moving_average', 'high_minus_low_relative','volatility', 'ln_volume']

# All Supported Ta-lib indicators: https://github.com/TA-Lib/ta-lib-python/blob/master/docs/funcs.md
TECHNICAL_INDICATORS = ['adx', 'adxr', 'apo', 'aroon_1','aroon_2', 'aroonosc',
 'bop', 'cci', 'cmo','dx', 'macd', 'macdsignal', 'macdhist', 'macd_ext',
 'macdsignal_ext', 'macdhist_ext', 'macd_fix', 'macdsignal_fix',
 'macdhist_fix', 'mfi', 'minus_di', 'mom', 'plus_di', 'dm', 'ppo',
 'roc', 'rocp', 'rocr', 'rocr100', 'rsi', 'slowk', 'slowd', 'fastk',
 'fastd', 'fastk_rsi', 'fastd_rsi', 'trix', 'ultosc', 'willr',
 'ad', 'adosc', 'obv', 'atr', 'natr', 'ht_dcperiod', 'ht_dcphase',
 'ht_phasor_inphase', 'ht_phasor_quadrature', 'ht_sine_sine', 'ht_sine_leadsine',
 'ht_trendmod', 'avgprice', 'medprice', 'typprice', 'wclprice']
TECHNICAL_PATTERNS = [g for g in df_full.keys() if g.find('cdl')>=0]
MACRO = ['gdppot_us_yoy', 'gdppot_us_qoq', 'cpi_core_yoy', 'cpi_core_mom', 'FEDFUNDS',
 'DGS1', 'DGS5', 'DGS10']
NUMERICAL = GROWTH + TECHNICAL_INDICATORS + TECHNICAL_PATTERNS + CUSTOM_NUMERICAL + MACRO

# CHECK: NO OTHER INDICATORS LEFT
OTHER = [k for k in df_full.keys() if k not in OHLCV + CATEGORICAL + NUMERICAL + TO_DROP]

In [6]:
# let's define on more custom numerical features
df_full['ln_volume'] = df_full.Volume.apply(lambda x: np.log(x))  #  if x > 0 else 0

/tmp/ipykernel_8564/3618675543.py:2: RuntimeWarning: divide by zero encountered in log
  df_full['ln_volume'] = df_full.Volume.apply(lambda x: np.log(x))  #  if x > 0 else 0


In [7]:
# tickers, min-max date, count of daily observations
# df_full.groupby(['Ticker'])['Date'].agg(['min','max','count'])

In [8]:
# truncated df_full with 25 years of data (and defined growth variables)
df = df_full[df_full.Date >= '2000-01-01']
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 182675 entries, 3490 to 5426
Columns: 203 entries, Open to ln_volume
dtypes: datetime64[ns](3), float64(129), int32(64), int64(5), object(2)
memory usage: 239.7+ MB


In [9]:
# let look at the features count and size:
df[NUMERICAL].info()

<class 'pandas.core.frame.DataFrame'>
Index: 182675 entries, 3490 to 5426
Columns: 184 entries, growth_1d to DGS10
dtypes: float64(121), int32(62), int64(1)
memory usage: 214.6 MB


## Question 1 (1 point): Dummies on Month and Week-of-Month

**Find the ABSOLUTE CORRELATION VALUE of the most correlated dummy <month-week_of_month> with the binary outcome variable `is_positive_growth_5d_future`?**

You saw in the correlation analysis and modeling that September and October may be important seasonal months. In this task, we'll go futher and try to generate dummies for Month and Week-of-month (starting from 1). For example, the first week of October should be coded similar to this: 'October_w1'.
Once you've generated the new set of variables, find the most correlated (in absolute value) one with `is_positive_growth_5d_future` and round it to 3 digits after the comma.

Suggested path to a solution:
- [[Source](https://stackoverflow.com/questions/25249033/week-of-a-month-pandas)] Use this formula to get the week of month for the datetime variable d: `(d.day-1)//7+1` 
- Define a new string variable for all month-week_of_month combinations. Append it to the CATEGORICAL features set. You should have 5 variables treated as CATEGORICAL now: 'Month', 'Weekday', 'Ticker', 'ticker_type', 'month_wom'. In the end, you should get 115 dummy features, including 60 (=12*5) week_month_of_week dummies.
- Use [pandas.get_dummies()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) to generate dummies.
- Use [pandas.DataFrame.corr()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) function (also used in [Code Snippet 1]) to get correlations with `is_positive_growth_5d_future`, filter out only variables representing the new dummy set, and sort it by absolute values (you can define a new column "abs_corr" in the dataframe with correlations), and find the highest value (among the new dummies features set).

**NOTE**: new dummies will be used as features in the next tasks, please leave them in the dataset.

In [10]:
# dummy variables are not generated from Date and numeric variables
df.loc[:,'Month'] = df.Month.dt.strftime('%B')
df.loc[:,'Weekday'] = df.Weekday.astype(str)

# generate week of month features
# df['month_wom'] = df.Date.dt.strftime('%B')+"_w"+((df.Date.dt.day - 1) // 7 + 1).astype(str)

# Use this formula to get the week of month for the datetime variable d: (d.day-1)//7+1
df.loc[:, 'month_wom']  = (df.Date.dt.day-1)//7+1

# Define a new string variable for all month-week_of_month combinations.
df.loc[:, 'month_wom']= df['Month'].astype(str) + '_w' + df['month_wom'].astype(str)

/tmp/ipykernel_8564/1581066930.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['January' 'January' 'January' ... 'May' 'May' 'May']' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.loc[:,'Month'] = df.Month.dt.strftime('%B')
/tmp/ipykernel_8564/1581066930.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0' '1' '2' ... '4' '0' '1']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df.loc[:,'Weekday'] = df.Weekday.astype(str)
/tmp/ipykernel_8564/1581066930.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
# Generate dummy variables (no need for bool, let's have int32 instead)
CATEGORICAL.append('month_wom')
CATEGORICAL

['Month', 'Weekday', 'Ticker', 'ticker_type', 'month_wom']

In [12]:
dummy_variables = pd.get_dummies(df[CATEGORICAL], dtype='int32')
dummy_variables.info()

<class 'pandas.core.frame.DataFrame'>
Index: 182675 entries, 3490 to 5426
Columns: 115 entries, Month_April to month_wom_September_w5
dtypes: int32(115)
memory usage: 81.5 MB


In [13]:
# get dummies names in a list
DUMMIES = dummy_variables.keys().to_list()

In [14]:
# Concatenate the dummy variables with the original DataFrame
df_with_dummies = pd.concat([df, dummy_variables], axis=1)
df_with_dummies[NUMERICAL+DUMMIES].info()

<class 'pandas.core.frame.DataFrame'>
Index: 182675 entries, 3490 to 5426
Columns: 299 entries, growth_1d to month_wom_September_w5
dtypes: float64(121), int32(177), int64(1)
memory usage: 294.8 MB


In [15]:
# correlation analysis with filtered out categories
corr_is_positive_growth_5d_future = df_with_dummies[DUMMIES+TO_PREDICT].corr()['is_positive_growth_5d_future']

# create a dataframe for an easy way to sort
corr_is_positive_growth_5d_future_df = pd.DataFrame(corr_is_positive_growth_5d_future)

In [16]:
# calculate the absolute value of the correlations
corr_is_positive_growth_5d_future_df['abs_corr'] = corr_is_positive_growth_5d_future_df['is_positive_growth_5d_future'].abs()

# filter the rows whose index is not in TO_PREDICT
filtered_df = corr_is_positive_growth_5d_future_df[~corr_is_positive_growth_5d_future_df.index.isin(TO_PREDICT)]

# sort by the absolute value of the correlation in descending order and select the first row
result_df = filtered_df.sort_values(by='abs_corr', ascending=False).head(1)

# Round it to 3 digits after the comma
result_df = result_df.round(3)

In [17]:
result_df

,is_positive_growth_5d_future,abs_corr
month_wom_September_w3,-0.035,0.035


## Question 2 (2 points): Define new "hand" rules on macro and technical indicators variables

**What is the precision score for the best of the NEW predictions (pred3 or pred4), rounded to 3 digits after the comma?**

Let's utilize the knowledge from the visualised tree (clf10) (Code Snippet 5: 1.4.4 Visualisation):

* You're asked to define two new 'hand' rules (leading to 'positive' subtrees): 
  - `pred3_manual_gdp_fastd`: (gdppot_us_yoy <= 0.027) & (fastd >= 0.251)
  - `pred4_manual_gdp_wti_oil`: (gdppot_us_yoy >= 0.027) & (growth_wti_oil_30d <= 1.005)

* Extend the Code Snippet 3 (Manual "hand rule" predictions): Calculate and add new rules (pred3 and pred4) to the dataframe.You should notice that one of the predictions doesn't have any positive predictions on TEST dataset (while it has many on TRAIN+VALIDATION). 

* Debug: check in the `new_df` and the original dataset/data generation process that we didn't make any mistakes during the data transformation step.

* Explain why this can happen even if there are no errors in the data features.

* As a result, write down the precision score for the remaining predictor (round to three decimal points). E.g. if you have 0.57897, your answer should be 0.579.

In [18]:
def temporal_split(df, min_date, max_date, train_prop=0.7, val_prop=0.15, test_prop=0.15):
    """
    Splits a DataFrame into three buckets based on the temporal order of the 'Date' column.

    Args:
        df (DataFrame): The DataFrame to split.
        min_date (str or Timestamp): Minimum date in the DataFrame.
        max_date (str or Timestamp): Maximum date in the DataFrame.
        train_prop (float): Proportion of data for training set (default: 0.6).
        val_prop (float): Proportion of data for validation set (default: 0.2).
        test_prop (float): Proportion of data for test set (default: 0.2).

    Returns:
        DataFrame: The input DataFrame with a new column 'split' indicating the split for each row.
    """
    # Define the date intervals
    train_end = min_date + pd.Timedelta(days=(max_date - min_date).days * train_prop)
    val_end = train_end + pd.Timedelta(days=(max_date - min_date).days * val_prop)

    # Assign split labels based on date ranges
    split_labels = []
    for date in df['Date']:
        if date <= train_end:
            split_labels.append('train')
        elif date <= val_end:
            split_labels.append('validation')
        else:
            split_labels.append('test')

    # Add 'split' column to the DataFrame
    df['split'] = split_labels

    return df

In [19]:
min_date_df = df_with_dummies.Date.min()
max_date_df = df_with_dummies.Date.max()

df_with_dummies = temporal_split(
    df_with_dummies,
    min_date = min_date_df,
    max_date = max_date_df,
)
df_with_dummies['split'].value_counts()/len(df_with_dummies)

split
train         0.675834
test          0.163290
validation    0.160876
Name: count, dtype: float64

In [20]:
# remove the "segmentation" problem (warning message on df performance after many joins and data transformations)
new_df = df_with_dummies.copy()

# Full dataframe (transformed and truncated to 25 years)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 182675 entries, 3490 to 5426
Columns: 320 entries, Open to split
dtypes: datetime64[ns](2), float64(129), int32(178), int64(5), object(6)
memory usage: 323.3+ MB


In [21]:
# check one record: it has abs. values, text, and numbers
new_df.head(1)

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,month_wom_October_w2,month_wom_October_w3,month_wom_October_w4,month_wom_October_w5,month_wom_September_w1,month_wom_September_w2,month_wom_September_w3,month_wom_September_w4,month_wom_September_w5,split
3490,58.6875,59.3125,56.0,58.28125,36.065567,53228400.0,MSFT,2000,January,0,...,0,0,0,0,0,0,0,0,0,train


In [22]:
# time split on train/validation/test: FIXED dates of split, approx. 70%, 15%, 15% split
new_df.groupby(['split'])['Date'].agg({'min','max','count'})

,min,count,max
split,,,
test,2020-09-14,29829,2024-05-07
train,2000-01-03,123458,2017-01-16
validation,2017-01-17,29388,2020-09-11


In [23]:
# what we try to predict
new_df[TO_PREDICT].head(1)

,growth_future_5d,is_positive_growth_5d_future
3490,0.963003,0


In [24]:
# to be used as features
new_df[NUMERICAL+DUMMIES].head(1)

,growth_1d,growth_3d,growth_7d,growth_30d,growth_90d,growth_365d,growth_dax_1d,growth_dax_3d,growth_dax_7d,growth_dax_30d,...,month_wom_October_w1,month_wom_October_w2,month_wom_October_w3,month_wom_October_w4,month_wom_October_w5,month_wom_September_w1,month_wom_September_w2,month_wom_September_w3,month_wom_September_w4,month_wom_September_w5
3490,0.998394,0.988341,0.991494,1.372333,1.222951,2.063053,0.970196,0.983855,1.051736,1.134572,...,0,0,0,0,0,0,0,0,0,0


In [25]:
# generate manual predictions
# Let's label all prediction features with prefix "pred"
new_df['pred0_manual_cci'] = (new_df.cci>200).astype(int)
new_df['pred1_manual_prev_g1'] = (new_df.growth_1d>1).astype(int)
new_df['pred2_manual_prev_g1_and_snp'] = ((new_df['growth_1d'] > 1) & (new_df['growth_snp500_1d'] > 1)).astype(int)

In [26]:
# TODO 2: find more "hand rules" - can get it from decision trees important factors, or randomly build on other most popular macro/tech indicators/ manual_features
new_df['pred3_manual_gdp_fastd'] = (new_df['gdppot_us_yoy'] <= 0.027) & (new_df['fastd'] >= 0.251).astype(int)
new_df['pred4_manual_gdp_wti_oil'] = (new_df['gdppot_us_yoy'] >= 0.027) & (new_df['growth_wti_oil_30d'] <= 1.005).astype(int)

In [27]:
new_df[['cci','growth_1d','growth_snp500_1d','pred0_manual_cci','pred1_manual_prev_g1','pred2_manual_prev_g1_and_snp','pred3_manual_gdp_fastd','pred4_manual_gdp_wti_oil','is_positive_growth_5d_future']]

,cci,growth_1d,growth_snp500_1d,pred0_manual_cci,pred1_manual_prev_g1,pred2_manual_prev_g1_and_snp,pred3_manual_gdp_fastd,pred4_manual_gdp_wti_oil,is_positive_growth_5d_future
3490,26.847237,0.998394,0.990451,0,0,0,False,False,0
3491,-34.319663,0.966220,0.961655,0,0,0,False,False,0
3492,-97.318008,1.010544,1.001922,0,1,1,False,False,0
3493,-169.947507,0.966502,1.000956,0,0,0,False,False,0
3494,-142.142685,1.013068,1.027090,0,1,1,False,False,1
...,...,...,...,...,...,...,...,...,...
5422,-29.424989,0.988994,0.984269,0,0,0,True,False,0
5423,-26.657181,1.001447,1.009128,0,1,1,True,False,0
5424,-123.785473,0.972302,1.012557,0,0,0,True,False,0
5425,-181.986224,0.989571,1.010326,0,0,0,True,False,0


In [28]:
PREDICTIONS = [k for k in new_df.keys() if k.startswith('pred')]
PREDICTIONS

['pred0_manual_cci',
 'pred1_manual_prev_g1',
 'pred2_manual_prev_g1_and_snp',
 'pred3_manual_gdp_fastd',
 'pred4_manual_gdp_wti_oil']

In [29]:
p = PREDICTIONS[0]
part1 = p.split('_')[0] # first prefix before '_'
print(f'Full column name: {p}, only first part: {part1}')

Full column name: pred0_manual_cci, only first part: pred0


In [30]:
# generate columns is_correct_
for pred in PREDICTIONS:
  part1 = pred.split('_')[0] # first prefix before '_'
  new_df[f'is_correct_{part1}'] =  (new_df[pred] == new_df.is_positive_growth_5d_future).astype(int)

In [31]:
# IS_CORRECT dataset
IS_CORRECT =  [k for k in new_df.keys() if k.startswith('is_correct_')]
IS_CORRECT

['is_correct_pred0',
 'is_correct_pred1',
 'is_correct_pred2',
 'is_correct_pred3',
 'is_correct_pred4']

In [32]:
new_df[PREDICTIONS+IS_CORRECT+['is_positive_growth_5d_future']]
len(new_df[new_df.split=='test'])

29829

In [33]:
# define "Precision" for ALL predictions on a Test dataset (~4 last years of trading)
for i,column in enumerate(IS_CORRECT):
  prediction_column = PREDICTIONS[i]
  is_correct_column = column
  filter = (new_df.split=='test') & (new_df[prediction_column]==1)
  print(f'Prediction column:{prediction_column} , is_correct_column: {is_correct_column}')
  print(new_df[filter][is_correct_column].value_counts())
  print(new_df[filter][is_correct_column].value_counts()/len(new_df[filter]))

  print('---------')

Prediction column:pred0_manual_cci , is_correct_column: is_correct_pred0
is_correct_pred0
1    455
0    344
Name: count, dtype: int64
is_correct_pred0
1    0.569462
0    0.430538
Name: count, dtype: float64
---------
Prediction column:pred1_manual_prev_g1 , is_correct_column: is_correct_pred1
is_correct_pred1
1    8621
0    6980
Name: count, dtype: int64
is_correct_pred1
1    0.552593
0    0.447407
Name: count, dtype: float64
---------
Prediction column:pred2_manual_prev_g1_and_snp , is_correct_column: is_correct_pred2
is_correct_pred2
1    5726
0    4729
Name: count, dtype: int64
is_correct_pred2
1    0.547681
0    0.452319
Name: count, dtype: float64
---------
Prediction column:pred3_manual_gdp_fastd , is_correct_column: is_correct_pred3
is_correct_pred3
1    16560
0    13262
Name: count, dtype: int64
is_correct_pred3
1    0.555295
0    0.444705
Name: count, dtype: float64
---------
Prediction column:pred4_manual_gdp_wti_oil , is_correct_column: is_correct_pred4
Series([], Name: coun

## Question 3 (1 point): Unique correct predictions from a 10-levels deep Decision Tree Classifier (pred5_clf_10) 

**What is the total number of records in the TEST dataset when the new prediction pred5_clf_10 is better than all 'hand' rules (pred0..pred4)?**

NOTE: please include `random_state=42` to Decision Tree Classifier init function (line `clf = DecisionTreeClassifier(max_depth=max_depth, random_state=42)`) to ensure everyone gets the same results.

Suggested solution:
* Step1: Rewrite the '1.4.3 Inference for a decision tree' piece for the Decision Tree Classifier with max_depth=10 (clf_10), so that you fit the model on TRAIN+VALIDATION sets (unchanged from the lecture), but predict on the whole set X_all (to be able to define a new column 'pred5_clf_10' in the dataframe new_df). Here is the [link](https://stackoverflow.com/questions/40729162/merging-results-from-model-predict-with-original-pandas-dataframe) with explanation. It will solve the problem in 1.4.5 when predictions were made only for Test dataset and couldn't be easily joined with the full dataset. 

* Step2: Once you have it, define a new column 'only_pred5_is_correct' similar to 'hand' prediction rules with several conditions: is_positive_growth_5d_future AND is_correct_pred5 should be equal 1, while all other predictions is_correct_pred0..is_correct_pred4 should be equal to 0.

* Step3: Convert 'only_pred5_is_correct' column from bool to int, and find how many times it is equal to 1 in the TEST set. Write down this as an answer.

ADVANCED: define a function that can be applied to the whole row of predictions ([a few examples of pandas-apply-row-functions](https://sparkbyexamples.com/pandas/pandas-apply-function-to-every-row/)) and can find whether some prediction 'predX' (where X is one of the predictions) is uniquely correct. It should work even if there are 100 predictions available, so that you don't define manually the condition for 'predX'.  

In [34]:
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

# set random state to 42
np.random.seed(42)


def remove_infinite_values(X):
    """
    Remove infinite values from the input array.

    Parameters:
    - X: Input array (NumPy array or array-like)

    Returns:
    - Array with infinite values removed
    """
    return X[np.isfinite(X).all(axis=1)]

In [35]:
# look carefully for 'count' to be close to total values (or you need to replace NaNs/remove NaNs), and min/max doesn't equal to -+inf.
#  it will give you an idea to dig deeper into some features to understand the 'nature' of a problem
pd.set_option('display.max_rows', None)

new_df[NUMERICAL+DUMMIES].describe().T

/home/clamytoe/miniconda3/envs/stocks/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,count,mean,std,min,25%,50%,75%,max
growth_1d,182660.0,1.000868e+00,2.356674e-02,4.601100e-01,9.906461e-01,1.000473e+00,1.010716e+00,3.018887e+00
growth_3d,182630.0,1.002558e+00,3.947590e-02,4.170567e-01,9.842405e-01,1.002120e+00,1.020217e+00,3.018887e+00
growth_7d,182570.0,1.005875e+00,5.838238e-02,4.065424e-01,9.774801e-01,1.005428e+00,1.033406e+00,3.018887e+00
growth_30d,182225.0,1.025093e+00,1.180397e-01,3.054439e-01,9.622796e-01,1.022171e+00,1.083990e+00,3.993310e+00
growth_90d,181325.0,1.077823e+00,2.209361e-01,1.890111e-01,9.587507e-01,1.061788e+00,1.175049e+00,5.970002e+00
growth_365d,176915.0,1.350498e+00,6.155275e-01,9.783037e-02,1.015438e+00,1.239138e+00,1.527952e+00,9.819906e+00
growth_dax_1d,179673.0,1.000258e+00,1.419365e-02,8.776139e-01,9.937851e-01,1.000761e+00,1.007159e+00,1.114020e+00
growth_dax_3d,179673.0,1.000813e+00,2.417672e-02,8.374862e-01,9.891694e-01,1.002138e+00,1.013879e+00,1.144124e+00
growth_dax_7d,179673.0,1.001993e+00,3.587574e-02,7.318924e-01,9.843275e-01,1.004785e+00,1.021502e+00,1.232460e+00
growth_dax_30d,179673.0,1.008638e+00,7.189179e-02,6.263172e-01,9.727150e-01,1.014873e+00,1.050698e+00,1.288371e+00


In [36]:
# Split the data into training and testing sets based on the split date
features_list = NUMERICAL+DUMMIES
to_predict = 'is_positive_growth_5d_future'

train_df = new_df[new_df.split.isin(['train','validation'])].copy(deep=True)
test_df = new_df[new_df.split.isin(['test'])].copy(deep=True)

# ONLY numerical Separate features and target variable for training and testing sets
# need Date and Ticker later when merging predictions to the dataset
X_train = train_df[features_list+[to_predict,'Date','Ticker']]
X_test = test_df[features_list+[to_predict,'Date','Ticker']]
X_all = new_df[features_list+[to_predict,'Date','Ticker']]

print(f'length: X_train {X_train.shape},  X_test {X_test.shape}, X_all {X_all.shape}')

length: X_train (152846, 302),  X_test (29829, 302), X_all (182675, 302)


In [37]:
# Can't have +-inf values . E.g. ln(volume)=-inf when volume==0 => substitute with 0

# Disable SettingWithCopyWarning
pd.options.mode.chained_assignment = None  # default='warn'

X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_all.replace([np.inf, -np.inf], np.nan, inplace=True)

# Need to fill NaNs somehow
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)
X_all.fillna(0, inplace=True)

print(f'length: X_train_imputed {X_train.shape},  X_test_imputed {X_test.shape}, X_all {X_all.shape}')

length: X_train_imputed (152846, 302),  X_test_imputed (29829, 302), X_all (182675, 302)


In [38]:
# you may want to remove 1-2% outliers based on percentile ==> not used here in Decision Trees
def remove_outliers_percentile(X, lower_percentile=1, upper_percentile=99):
    """
    Remove outliers from the input array based on percentiles.

    Parameters:
    - X: Input array (NumPy array or array-like)
    - lower_percentile: Lower percentile threshold (float, default=1)
    - upper_percentile: Upper percentile threshold (float, default=99)

    Returns:
    - Array with outliers removed
    """
    lower_bound = np.percentile(X, lower_percentile, axis=0)
    upper_bound = np.percentile(X, upper_percentile, axis=0)
    mask = np.logical_and(np.all(X >= lower_bound, axis=1), np.all(X <= upper_bound, axis=1))
    
    return X[mask]

In [39]:
X_train_imputed = X_train # we won't use outliers removal to save more data to train: remove_outliers_percentile(X_train)
X_test_imputed = X_test # we won't use outliers removal to save more data to test: remove_outliers_percentile(X_test)
X_all_imputed = X_all # we won't use outliers removal to save more data to test: remove_outliers_percentile(X_all)

In [40]:
# same shape
print(f'length: X_train_imputed {X_train_imputed.shape},  X_test_imputed {X_test_imputed.shape}, X_all_imputed {X_all_imputed.shape}')

length: X_train_imputed (152846, 302),  X_test_imputed (29829, 302), X_all_imputed (182675, 302)


In [41]:
y_train = X_train_imputed[to_predict]
y_test = X_test_imputed[to_predict]
y_all = X_all_imputed[to_predict]

# remove y_train, y_test from X_ dataframes
del X_train_imputed[to_predict]
del X_test_imputed[to_predict]
del X_all_imputed[to_predict]

In [42]:
# estimation/fit function (using dataframe of features X and what to predict y) --> optimising total accuracy
# max_depth is hyperParameter
def fit_decision_tree(X, y, max_depth=10):
    # Initialize the Decision Tree Classifier
    clf = DecisionTreeClassifier(
        max_depth=max_depth,
        random_state=42,
    )
    
    # Fit the classifier to the training data
    clf.fit(X, y)
    
    return clf, X.columns

In [43]:
%%time
clf_10, train_columns = fit_decision_tree(
    X=X_all_imputed.drop(['Date','Ticker'],axis=1),
    y=y_all,
)

CPU times: user 20.6 s, sys: 140 ms, total: 20.7 s
Wall time: 20.7 s


In [44]:
# TODO 3: TRAIN only on train dataset, experiment with trees with depth 1..20 --> find the best one on VALID dataset
#       for the "best" tree model: find precision on the TEST set
def predict_decision_tree(clf:DecisionTreeClassifier, df_X:pd.DataFrame, y_true: pd.Series):
    # Predict the target variable on the test data
    y_pred = clf.predict(df_X)
    
    max_depth = clf.tree_.max_depth
    
    # Print the maximum depth
    print("Maximum depth of the decision tree:", max_depth)
    
    # Calculate the accuracy/precision of the model
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    print(f'Accuracy = {accuracy}, precision = {precision}')
    
    # resulting df
    result_df = pd.concat([df_X, y_true, pd.Series(y_pred, index=df_X.index, name='pred_')], axis=1)

    return result_df

In [45]:
pred_10 = predict_decision_tree(clf_10, X_all_imputed.drop(['Date','Ticker'],axis=1), y_all)

Maximum depth of the decision tree: 10
Accuracy = 0.612278636923498, precision = 0.623247718911281


In [46]:
pred_10.tail()

,growth_1d,growth_3d,growth_7d,growth_30d,growth_90d,growth_365d,growth_dax_1d,growth_dax_3d,growth_dax_7d,growth_dax_30d,...,month_wom_October_w3,month_wom_October_w4,month_wom_October_w5,month_wom_September_w1,month_wom_September_w2,month_wom_September_w3,month_wom_September_w4,month_wom_September_w5,is_positive_growth_5d_future,pred_
5422,0.988994,0.984120,1.021587,1.015755,1.046955,1.793402,0.989726,1.000831,1.010983,0.999750,...,0,0,0,0,0,0,0,0,0,0
5423,1.001447,0.998419,0.996346,0.997589,1.031967,1.790292,0.998011,0.985435,1.001999,0.997982,...,0,0,0,0,0,0,0,0,0,0
5424,0.972302,0.962991,0.969433,0.989441,1.002348,1.738504,1.005873,0.993558,0.992499,1.000784,...,0,0,0,0,0,0,0,0,0,0
5425,0.989571,0.963553,0.952804,0.972509,0.989811,1.732523,1.009644,1.013553,1.004783,1.008886,...,0,0,0,0,0,0,0,0,0,0
5426,0.989735,0.952285,0.938518,0.978882,1.002706,1.737153,1.014021,1.029813,1.028619,1.013796,...,0,0,0,0,0,0,0,0,0,0


In [47]:
# Predictions of a decision tree of depth "10"
pred_10.pred_.value_counts()

pred_
1    116063
0     66612
Name: count, dtype: int64

In [48]:
pred5_clf_10 = pred_10[['pred_']].rename(columns={'pred_': 'pred_tree_clf10'})
pred5_clf_10.head(1)

,pred_tree_clf10
3490,1


In [49]:
# define column 'pred5_clf_10' in new_df with the results from pred10_df['pred_tree_clf10']
new_df['pred5_clf_10'] = pred5_clf_10['pred_tree_clf10']
new_df.head()

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,pred1_manual_prev_g1,pred2_manual_prev_g1_and_snp,pred3_manual_gdp_fastd,pred4_manual_gdp_wti_oil,is_correct_pred0,is_correct_pred1,is_correct_pred2,is_correct_pred3,is_correct_pred4,pred5_clf_10
3490,58.68750,59.3125,56.00000,58.28125,36.065567,53228400.0,MSFT,2000,January,0,...,0,0,False,False,1,1,1,1,1,1
3491,56.78125,58.5625,56.12500,56.31250,34.847271,54119000.0,MSFT,2000,January,1,...,0,0,False,False,1,1,1,1,1,1
3492,55.56250,58.1875,54.68750,56.90625,35.214706,64059600.0,MSFT,2000,January,2,...,1,1,False,False,1,0,0,1,1,1
3493,56.09375,56.9375,54.18750,55.00000,34.035072,54976600.0,MSFT,2000,January,3,...,0,0,False,False,1,1,1,1,1,1
3494,54.31250,56.1250,53.65625,55.71875,34.479843,62013600.0,MSFT,2000,January,4,...,1,1,False,False,0,1,1,0,0,1


In [50]:
# new column is_correct_pred5
new_df['is_correct_pred5'] = (new_df['pred5_clf_10'] == new_df['is_positive_growth_5d_future']).astype(int)

print(f'Prediction column: pred5_clf_10 , is_correct_column: is_correct_pred5')
print(new_df['is_correct_pred5'].value_counts())

Prediction column: pred5_clf_10 , is_correct_column: is_correct_pred5
is_correct_pred5
1    111848
0     70827
Name: count, dtype: int64


In [51]:
# new column 'only_pred5_is_correct'
new_df['only_pred5_is_correct'] = (new_df.is_positive_growth_5d_future == 1) & (new_df.is_correct_pred5 == 1) & (new_df.is_correct_pred0 == 0) & (new_df.is_correct_pred1 == 0) & (new_df.is_correct_pred2 == 0) & (new_df.is_correct_pred3 == 0) & (new_df.is_correct_pred4 == 0)
new_df['only_pred5_is_correct'] = new_df['only_pred5_is_correct'].astype(int)

In [52]:
print(f'Prediction column: only_pred5_is_correct')
print(new_df['only_pred5_is_correct'].value_counts())

Prediction column: only_pred5_is_correct
only_pred5_is_correct
0    181399
1      1276
Name: count, dtype: int64


In [53]:
new_df[new_df.split=="test"]["only_pred5_is_correct"].value_counts()

only_pred5_is_correct
0    29828
1        1
Name: count, dtype: int64

In [54]:
def unique_correct(row, correct_col):
    correct_value = row[correct_col]
    pred_columns = [col for col in row.index if col.startswith('pred')]
    correct_preds = [col for col in pred_columns if row[col] == correct_value]

    results = {}
    for col in pred_columns:
        results[f'only_{col}_is_correct'] = 1 if (len(correct_preds) == 1 and correct_preds[0] == col) else 0

    return pd.Series(results)

In [55]:
df_unique = new_df.copy()
correct_col = 'is_positive_growth_5d_future'
results = df_unique.apply(lambda row: unique_correct(row, correct_col), axis=1)

# add the results to the dataframe copy
df_unique = pd.concat([df_unique, results], axis=1)

In [56]:
df_unique.columns.difference(new_df.columns)

Index(['only_pred0_manual_cci_is_correct',
       'only_pred1_manual_prev_g1_is_correct',
       'only_pred2_manual_prev_g1_and_snp_is_correct',
       'only_pred3_manual_gdp_fastd_is_correct',
       'only_pred4_manual_gdp_wti_oil_is_correct',
       'only_pred5_clf_10_is_correct'],
      dtype='object')

## Question 4: (2 points) Hyperparameter tuning for a Decision Tree

**What is the optimal tree depth (from 1 to 20) for a DecisionTreeClassifier?**

NOTE: please include `random_state=42` to Decision Tree Classifier init function (line `clf = DecisionTreeClassifier(max_depth=max_depth, random_state=42)`) to ensure consistency in results.

Follow these steps to find the optimal `max_depth`:
* Iterate through `max_depth` values from 1 to 20.
* Train the Decision Tree Classifier with the current `max_depth` parameter.
* Optionally, visualize how the 'head' of each fitted tree changes with more advanced (=deep) trees. You can use the [`sklearn.tree.plot_tree()`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html) function, or the compact way  with the `export_text()` functionality ([Stack Overflow example](https://stackoverflow.com/questions/20156951/how-do-i-find-which-attributes-my-tree-splits-on-when-using-scikit-learn)):
  ```
  from sklearn.tree import export_text
  tree_rules = export_text(model, feature_names=list(X_train), max_depth=3)
  print(tree_rules)
  ```
* Calculate the precision score (you can use the function [sklearn.metrics.precision_score()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html)) on the TEST dataset for each of the fitted trees. You can also compare it with the precision score on a VALIDATION dataset, which is included to the training phase (to have more data to train on). You should see that the precision score on a VALIDATION set starts to grow with the complexity of a tree (overfit), which isn't seen on the precision score on TEST.
* Identify the optimal `max_depth`, where the  precision score is the highest on the TEST dataset. Record this value as  **best_max_depth** and submit as an answer.
* Make predictions on all records (TRAIN+VALIDATION+TEST) and add the new prediction `pred6_clf_best` to the dataframe `new_df`.

Additionally, compare the precision score of the tuned decision tree with previous predictions. You should observe an improvement (>0.58, or more than 58% precision), indicating that the tuned tree outperforms previous manual "hand" rules and Decision Tree predictions.

ADVANCED: Read more about different aspects of [scikit-learn Decision Trees](https://scikit-learn.org/stable/modules/tree.html). Draw a line of precision/accuracy vs. max_depth and note whether there's a saturation point of precision/accuracy as max_depth increases. In theory, there should be a trade-off between better fitting (=more complex trees) and generalization.

In [57]:
from sklearn.tree import export_text
from tqdm import tqdm

In [58]:
# range of max_depth values
max_depths = range(1, 22)

# initialize a list to store the scores
scores = []

# loop over the range of max_depth values
for max_depth in tqdm(max_depths):
    clf, train_columns = fit_decision_tree(
        X=X_train_imputed.drop(['Date','Ticker'],axis=1),
        y=y_train,
        max_depth=max_depth,
    )

    # predict on test set
    y_pred = clf.predict(X_test.drop(['Date','Ticker'],axis=1))

    # calculate precision
    precision = precision_score(y_test, y_pred)
    scores.append((max_depth, precision))

    # (view tree structure
    if max_depth <= 3:
        tree_rules = export_text(
            clf, 
            feature_names=list(train_columns), 
            max_depth=3,
        )
        print(f"Tree rules for max_depth={max_depth}:\n", tree_rules)

  5%|████                                                                                | 1/21 [00:01<00:38,  1.94s/it]

Tree rules for max_depth=1:
 |--- gdppot_us_yoy <= 0.03
|   |--- class: 1
|--- gdppot_us_yoy >  0.03
|   |--- class: 0



 10%|████████                                                                            | 2/21 [00:05<00:55,  2.90s/it]

Tree rules for max_depth=2:
 |--- gdppot_us_yoy <= 0.03
|   |--- fastd <= 0.25
|   |   |--- class: 0
|   |--- fastd >  0.25
|   |   |--- class: 1
|--- gdppot_us_yoy >  0.03
|   |--- growth_wti_oil_30d <= 1.00
|   |   |--- class: 1
|   |--- growth_wti_oil_30d >  1.00
|   |   |--- class: 0



 14%|████████████                                                                        | 3/21 [00:10<01:13,  4.06s/it]

Tree rules for max_depth=3:
 |--- gdppot_us_yoy <= 0.03
|   |--- fastd <= 0.25
|   |   |--- growth_brent_oil_90d <= 1.06
|   |   |   |--- class: 0
|   |   |--- growth_brent_oil_90d >  1.06
|   |   |   |--- class: 1
|   |--- fastd >  0.25
|   |   |--- growth_gold_365d <= 1.53
|   |   |   |--- class: 1
|   |   |--- growth_gold_365d >  1.53
|   |   |   |--- class: 0
|--- gdppot_us_yoy >  0.03
|   |--- growth_wti_oil_30d <= 1.00
|   |   |--- growth_dji_90d <= 0.97
|   |   |   |--- class: 1
|   |   |--- growth_dji_90d >  0.97
|   |   |   |--- class: 0
|   |--- growth_wti_oil_30d >  1.00
|   |   |--- Month_September <= 0.50
|   |   |   |--- class: 0
|   |   |--- Month_September >  0.50
|   |   |   |--- class: 0



100%|███████████████████████████████████████████████████████████████████████████████████| 21/21 [06:00<00:00, 17.18s/it]


In [60]:
# find optimal depth
best_max_depth, best_precision = max(scores, key=lambda x: x[1])
print(f"The optimal depth of the tree is: {best_max_depth}, with a precision of: {best_precision:.4f}")

The optimal depth of the tree is: 15, with a precision of: 0.5864


## [EXPLORATORY] Question 5: What data is missing? 

Now that you have some insights from the correlation analysis and the Decision Trees regarding the most influential variables, suggest new indicators you would like to include in the dataset and explain why.

You can also propose something entirely different based on your intuition, but it should be relevant to the shared dataset of the largest Indian, EU, and US stocks. If you choose this approach, please specify the data source as well.

Depending on the investment, it would be nice to have a (daily, 4-hour, 15-min) or (1-hour, 15-min, 5-min) trend indicator. Then if all time frames indicate an up or downtrend, then a correspsonding buy or sell signal could be indicated and a jump in point could be found to jump into the trade.